In [ ]:
# importing Neccessary Library and constant variable

!pip install tf_clahe
!pip install -U scikit-learn
!pip install matplotlib
!pip install pandas

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import tf_clahe
from glob import glob
from tqdm import tqdm
import os


IMG_H = 128
IMG_W = 128
IMG_C = 3  ## Change this to 1 for grayscale.

In [ ]:
'''
    Function for Balance Contrast Enhancement Technique (BCET)
    This technique provides solution to biased color (RGB) composition. 
    The contrast of the image can be stretched or compressed without changing the histogram pattern of the input image(x).
    The solution is based on the parabolic function obtained from the input image.
'''
@tf.function
def bcet(img):

    
    Lmin = tf.reduce_min(img) # MINIMUM OF INPUT IMAGE
#     Lmin = np.min(img) # MINIMUM OF INPUT IMAGE
#     print("Lmin", Lmin)
    Lmax = tf.reduce_max(img) # MAXIMUM OF INPUT IMAGE
#     Lmax = np.max(img) # MAXIMUM OF INPUT IMAGE
#     print("Lmax", Lmax)
    Lmean = tf.reduce_mean(img) #MEAN OF INPUT IMAGE
#     Lmean = np.mean(img) #MEAN OF INPUT IMAGE
#     print("Lmean", Lmean)
    LMssum = tf.reduce_mean(img * img) #MEAN SQUARE SUM OF INPUT IMAGE
#     LMssum = np.mean(pow(img, 2)) #MEAN SQUARE SUM OF INPUT IMAGE
#     print("LMssum", LMssum)

    Gmin = tf.constant(0, dtype="float32") #MINIMUM OF OUTPUT IMAGE
    Gmax = tf.constant(255, dtype="float32") #MAXIMUM OF OUTPUT IMAGE
    Gmean = tf.constant(110, dtype="float32") #MEAN OF OUTPUT IMAGE
    
    subber = tf.constant(2, dtype="float32")

    bnum = Lmax * Lmax *(Gmean-Gmin) - LMssum*(Gmax-Gmin) + Lmin * Lmin *(Gmax-Gmean)
    bden = subber*(Lmax*(Gmean-Gmin)-Lmean*(Gmax-Gmin)+Lmin*(Gmax-Gmean))

    b = bnum/bden

    a = (Gmax-Gmin)/((Lmax-Lmin)*(Lmax+Lmin-subber*b))

    c = Gmin - a*(Lmin-b) * (Lmin-b)

    y = a * (img-b) * (img-b) + c #PARABOLIC FUNCTION

    return y

def bcet_processing(img,channels=3):
    img = tf.cast(img, tf.float32)
    layers = []
    for i in range(channels):
        layer = img[:,:,i]
        layer = bcet(layer)
        layers.append(layer)
        
    final_image = tf.stack(layers, axis=-1)

    return final_image

In [ ]:
# function for  preprocessing data 
def prep_stage(x):
    ### implement clahe to images
    x = tf_clahe.clahe(x)
    
    ### implement BCET to iamges
    x = bcet_processing(x)
    
    ### crop or pad images
    x = tf.image.resize_with_crop_or_pad(x, IMG_H, IMG_W)
    return x



def augment_dataset_batch_test(dataset_batch):
    AUTOTUNE = tf.data.AUTOTUNE
    
    
#     dataset_batch = dataset_batch.map(lambda x, y: (tf.image.grayscale_to_rgb(x), y))
    
    dataset_batch = dataset_batch.map(lambda x, y: (tf.image.per_image_standardization(x), y), 
              num_parallel_calls=AUTOTUNE)
    
    
    return dataset_batch

In [ ]:
def read_data_with_labels(filepath, class_names):
    image_list = []
    label_list = []
    for class_n in class_names:  # do dogs and cats
        path = os.path.join(filepath,class_n)  # create path to dogs and cats
        class_num = class_names.index(class_n)  # get the classification  (0 or a 1). 0=dog 1=cat

        for img in tqdm(os.listdir(path)):  
            if ".DS_Store" != img:
                filpath = os.path.join(path,img)
#                 print(filpath, class_num)
                image_list.append(filpath)
                label_list.append(class_num)
#     print(image_list, label_list)
    return image_list, label_list


def load_image_with_label(image_path, label):
    class_names = ["normal", "defect"]
#     print(image_path)
    img = tf.io.read_file(image_path)
    img = tf.io.decode_bmp(img, channels=IMG_C)
    img = prep_stage(img)
    img = tf.cast(img, tf.float32)
#     img = (img - 127.5) / 127.5
    
    return img, label



def tf_dataset_labels(images_path, batch_size, class_names=None):
    
    filenames, labels = read_data_with_labels(images_path, class_names)
    sizeoffiles = len(filenames)
#     print("testing")
#     print(filenames, labels)
    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.shuffle(buffer_size=10240)
    dataset = dataset.map(load_image_with_label, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    dataset = dataset.batch(sizeoffiles)
    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return dataset

In [ ]:
# load image dataset for testing with labels
def load_dataset(filename, class_name, size=(IMG_H,IMG_W)):
	# load image with the preferred size
    pixels = tf_dataset_labels(images_path=filename, batch_size=25, class_names=class_name)
#     pixels = augment_dataset_batch_test(pixels)
    
    return pixels

In [ ]:
if __name__ == "__main__":
    
    '''
    In Default:
    Clahe: OFF
    BCET: OFF
    Resize: crop or padding (decided by tensorflow)
    Datasets: For trainning dataset, it'll have additional datasets (flip-up-down and flip-right-left)
    '''
    
    
    # run the function here
    nameModel= "numbers_normal"
    print("start: ", nameModel)
    """ Set Hyperparameters """
    batch_size = 25
    num_epochs = 2
    resume_trainning = False
    
    # set dir of files
    train_data_path = "classification_data/train_data"
    test_data_path = "classification_data/test_data"

    input_shape = (IMG_W, IMG_H, IMG_C)
    # print(input_shape)

    # set input 
    inputs = tf.keras.layers.Input(input_shape, name="input_1")
    
    class_names = ["normal", "defect"] # normal = 0, defect = 1
    dataset_train = load_dataset(train_data_path, class_names)
    dataset_test = load_dataset(test_data_path, class_names)
    
 
    train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.3) # don't use rescale = 1./255

    train_generator = train_datagen.flow_from_directory(train_data_path,
                                                     target_size=(IMG_W,IMG_H),
                                                     batch_size=batch_size,
                                                     shuffle=True,
                                                     class_mode='categorical',
                                                     subset='training')

    validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split=0.3)

    validation_generator =  validation_datagen.flow_from_directory(train_data_path,
                                                                target_size=(IMG_W,IMG_H),
                                                     batch_size=batch_size,
                                                                class_mode='categorical',
                                                                subset='validation')  

    
    image_batch_train, labels_batch_train = next(iter(train_generator))
    print(image_batch_train.shape)
    print(labels_batch_train.shape)


    
    data_augmentation = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
      tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
    ])
    
    
    
    model_res50 = tf.keras.models.Sequential()

    model_res50.add(tf.keras.applications.ResNet50(
        include_top=False,
        pooling='avg',
        weights='imagenet'
        ))

    model_res50.add(tf.keras.layers.Dense(2, activation='softmax'))

    model_res50.layers[0].trainable = False 

    model_res50.summary()
    
    base_learning_rate = 0.0001
    
    model_res50.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(lr=base_learning_rate/10),
              metrics=['accuracy'])

    fit_history = model_res50.fit(
        train_generator,
        epochs=num_epochs,
        validation_data=validation_generator
    )
    
    
    
    acc = fit_history.history['accuracy']
    val_acc = fit_history.history['val_accuracy']
    loss = fit_history.history['loss']
    val_loss = fit_history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'r', label='Training accuracy')
    plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
    plt.title('Training and validation accuracy')

    plt.figure()

    plt.plot(epochs, loss, 'r', label='Training Loss')
    plt.plot(epochs, val_loss, 'b', label='Validation Loss')
    plt.title('Training and validation loss')

    plt.legend()

    plt.show()

#     loss, accuracy = model.evaluate(dataset_test)
#     print('Test accuracy :', accuracy)
    # for images, labels in dataset_train:
    #     print(images.shape, labels.shape)